# Assignment IV Answer

import the packages

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import train_test_split

load the data

In [46]:
from ISLP import load_data

In [47]:
data=load_data('Default')
data

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


### 1.	We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the Default data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefcients in two diferent ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the sm.GLM() function. Do not forget to set a random seed before beginning your analysis. 

#### (a) Using the summarize() and sm.GLM() functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors. 

In [48]:
x=data[['income','balance']]
y=data['default']=="Yes"
x_i = MS(x).fit_transform(x)
model=sm.GLM(y,x_i,family=sm.families.Binomial()).fit()
print(model.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -789.48
Date:                Mon, 22 Sep 2025   Deviance:                       1579.0
Time:                        21:38:02   Pearson chi2:                 6.95e+03
No. Iterations:                     9   Pseudo R-squ. (CS):             0.1256
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    -11.5405      0.435    -26.544      0.0

#### (b) Write a function, boot_fn(), that takes as input the Default data set as well as an index of the observations, and that outputs the coefcient estimates for income and balance in the multiple logistic regression model. 

In [49]:
def boot_fn(D, idx):
    data = D.loc[idx]
    x = data[['income', 'balance']]
    y = data['default'] == "Yes"
    x_i = MS(x).fit_transform(x)
    model = sm.GLM(y, x_i, family=sm.families.Binomial()).fit()
    return model.params

#### (c) Following the bootstrap example in the lab, use your boot_fn() function to estimate the standard errors of the logistic regression coefcients for income and balance. 

In [50]:
B = 1000
first_=0
second_ = 0
data = load_data('Default')
for _ in range(B):
    idx = np.random.choice(data.shape[0], 1000, replace=True)
    value = boot_fn(data, idx)
    first_ += value
    second_ += value**2
SE = np.sqrt(second_ / B - (first_ / B)**2)

print("Bootstrap estimated standard errors:")
print(SE[['income', 'balance']])

Bootstrap estimated standard errors:
income     0.000016
balance    0.000810
dtype: float64


#### (d) Comment on the estimated standard errors obtained using the sm.GLM() function and using the bootstrap.

Answer:

The standard errors from sm.GLM() and the bootstrap are very close.

This suggests the model fits the data well and assumptions are reasonable.

Both methods show 'balance' is a much stronger predictor than 'income'.

# .................................................................................................................................................................................................

### 2.	We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis. 

#### (a) Fit a logistic regression model that uses income and balance to predict default. 

In [55]:
x=data[['income','balance']]
y=data['default']=="yes"
x_i = MS(x).fit_transform(x)
model=sm.GLM(y,x_i,family=sm.families.Binomial()).fit()
print(model.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4444e-09
Date:                Mon, 22 Sep 2025   Deviance:                   2.8910e-09
Time:                        21:38:58   Pearson chi2:                 1.44e-09
No. Iterations:                    28   Pseudo R-squ. (CS):                nan
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    -29.5661    5.5e+04     -0.001      1.0

C:\Users\HP-15s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\genmod\generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\HP-15s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\genmod\generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\HP-15s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\genmod\generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warn

#### (b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps: 

i. Split the sample set into a training set and a validation set. 


ii. Fit a multiple logistic regression model using only the training observations. 


iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5. 

iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified. 

In [56]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_tr_i = MS(x_train).fit_transform(x_train)
x_te_i = MS(x_test).fit_transform(x_test)
model=sm.GLM(y_train,x_tr_i,family=sm.families.Binomial()).fit()
probs=model.predict(x_te_i)
pred = probs > 0.5
np.mean(pred != y_test)  

C:\Users\HP-15s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\genmod\generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\HP-15s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\genmod\generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\HP-15s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\genmod\generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warn

np.float64(0.0)

#### (c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

In [57]:
err=[]
for seed in [3,4,5]:
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=seed)
    x_tr_i = MS(x_train).fit_transform(x_train)
    x_te_i = MS(x_test).fit_transform(x_test)
    model=sm.GLM(y_train,x_tr_i,family=sm.families.Binomial()).fit()
    probs=model.predict(x_te_i)
    pred = probs > 0.5
    err_3=np.mean(pred != y_test)  
    err.append(err_3)
print(err)

[np.float64(0.0), np.float64(0.0), np.float64(0.0)]


C:\Users\HP-15s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\genmod\generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\HP-15s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\genmod\generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\HP-15s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\genmod\generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warn